In [13]:
import json
import uuid
import boto3
import s3fs
import rasterio
import xarray as xr
import rioxarray
from pystac_client import Client
import IPython.display
from IPython.display import display_html, display_javascript
from copy import deepcopy
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape
import hvplot.pandas  # noqa: F401

In [ ]:
# Install: cartopy, geoviews

# STAC Metadata Visualizations

### Client

In [2]:
URL = "https://planetarycomputer.microsoft.com/api/stac/v1"
headers = []

cat = Client.open(URL, headers=headers)
cat

<Client id=microsoft-pc>

### Search

In [3]:
# AOI around Delfzijl, in northern Netherlands
geom = {
    "type": "Polygon",
    "coordinates": [
        [
            [6.42425537109375, 53.174765470134616],
            [7.344360351562499, 53.174765470134616],
            [7.344360351562499, 53.67393435835391],
            [6.42425537109375, 53.67393435835391],
            [6.42425537109375, 53.174765470134616],
        ]
    ],
}

# limit sets the # of items per page so we can see multiple pages getting fetched
search = cat.search(
    max_items=50,
    collections="aster-l1t",
    intersects=geom,
    datetime="2000-01-01/2010-12-31",
)

# retrieve the items as dictionaries, rather than Item objects
items = list(search.items_as_dicts())
len(items)

50

### GeoPandas

In [5]:
# convert a list of STAC Items into a GeoDataFrame
def items_to_geodataframe(items):
    _items = []
    for i in items:
        _i = deepcopy(i)
        _i["geometry"] = shape(_i["geometry"])
        _items.append(_i)
    gdf = gpd.GeoDataFrame(pd.json_normalize(_items))
    for field in ["properties.datetime", "properties.created", "properties.updated"]:
        if field in gdf:
            gdf[field] = pd.to_datetime(gdf[field])
    gdf.set_index("properties.datetime", inplace=True)
    return gdf

In [6]:
# convert geometry to a GeoDataFrame
aoi_gdf = gpd.GeoDataFrame([{"geometry": shape(geom)}])
aoi_gdf

,geometry
0,"POLYGON ((6.42426 53.17477, 7.34436 53.17477, ..."


In [8]:
# convert found items to a GeoDataFrame
items_gdf = items_to_geodataframe(items)
items_gdf.head(3)

,id,bbox,type,links,geometry,collection,stac_extensions,stac_version,assets.TIR.href,assets.TIR.type,...,assets.vnir-browse.href,assets.vnir-browse.type,assets.vnir-browse.roles,assets.vnir-browse.title,assets.vnir-browse.description,assets.qa-txt.href,assets.qa-txt.type,assets.qa-txt.roles,assets.qa-txt.title,assets.qa-txt.description
properties.datetime,,,,,,,,,,,,,,,,,,,,,
2006-12-14 21:25:39.910000+00:00,AST_L1T_00312142006212539_20150517105406,"[5.8893619, 53.006463, 7.079664, 53.6760341]",Feature,"[{'rel': 'collection', 'type': 'application/js...","POLYGON ((6.83853 53.67589, 6.84669 53.67603, ...",aster-l1t,[https://stac-extensions.github.io/eo/v1.0.0/s...,1.0.0,https://astersa.blob.core.windows.net/aster/im...,image/tiff; application=geotiff; profile=cloud...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-09-19 10:50:24.192000+00:00,AST_L1T_00309192006105024_20150516061550,"[6.2313306, 52.613738, 7.4219194, 53.2988665]",Feature,"[{'rel': 'collection', 'type': 'application/js...","POLYGON ((7.42138 53.15465, 7.42093 53.15377, ...",aster-l1t,[https://stac-extensions.github.io/eo/v1.0.0/s...,1.0.0,https://astersa.blob.core.windows.net/aster/im...,image/tiff; application=geotiff; profile=cloud...,...,https://astersa.blob.core.windows.net/aster/im...,image/jpeg,[thumbnail],VNIR browse file,Standalone reduced resolution VNIR,NaN,NaN,NaN,NaN,NaN
2006-09-19 10:50:15.355000+00:00,AST_L1T_00309192006105015_20150516061540,"[6.4862887, 53.1318137, 7.6929281, 53.8181412]",Feature,"[{'rel': 'collection', 'type': 'application/js...","POLYGON ((7.69274 53.67237, 7.69231 53.67153, ...",aster-l1t,[https://stac-extensions.github.io/eo/v1.0.0/s...,1.0.0,https://astersa.blob.core.windows.net/aster/im...,image/tiff; application=geotiff; profile=cloud...,...,https://astersa.blob.core.windows.net/aster/im...,image/jpeg,[thumbnail],VNIR browse file,Standalone reduced resolution VNIR,https://astersa.blob.core.windows.net/aster/im...,text/plain,[metadata],QA browse file,Geometric quality assessment report.


### Plot Geometries on a Map

In [14]:
# plot polygons on a map with background tiles.
def plot_polygons(data, *args, **kwargs):
    return data.hvplot.polygons(
        *args,
        geo=True,
        projection="GOOGLE_MERCATOR",
        xaxis=None,
        yaxis=None,
        frame_width=600,
        frame_height=600,
        fill_alpha=0,
        line_width=4,
        **kwargs,
    )

In [15]:
plot_polygons(aoi_gdf, tiles="OSM", line_color="red")

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [16]:
plot_polygons(items_gdf, tiles="OSM")

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [17]:
plot_polygons(items_gdf, tiles="OSM") * plot_polygons(aoi_gdf, line_color="red")

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]
   .Polygons.II :Polygons   [Longitude,Latitude]

### Line Plots

In [18]:
items_df = pd.DataFrame(items_gdf)

plot_fields = [
    "properties.eo:cloud_cover",
    "properties.view:sun_azimuth",
    "properties.view:sun_elevation",
]

items_df[plot_fields].hvplot(height=500, width=800).opts(legend_position="top_right")

:NdOverlay   [Variable]
   :Curve   [properties.datetime]   (value)